# Project A/B Testing for ShoeFly

##### This project showcases an exploratory data analysis (EDA) and an extract-transform-load (ETL) process on an advertisement click dataset. It begins by loading and inspecting the data to understand its structure, identify missing values, and clean inconsistencies. Key insights are derived by analyzing user interactions with ads across different sources and experimental groups, calculating metrics like click-through rates (CTR) and identifying trends over time. The process involves feature engineering (e.g., creating an is_click column), data aggregation (e.g., grouping by ad source and day), and pivoting tables to visualize click patterns. Additional steps include handling missing data, calculating overall CTR, and saving the cleaned dataset for further use. This project highlights the ability to extract actionable insights and prepare data for advanced analysis or visualization.

Import necessary libraries


In [1]:
import pandas as pd

### 1. Data Extraction: Load the dataset


In [2]:
ad_clicks = pd.read_csv('ad_clicks.csv')

### 2. Data Preview: Initial Exploration


###### for better visualization, we will use the display() methond instead of print()

In [3]:
print("Dataset Preview:")
display(ad_clicks.head())

Dataset Preview:


,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


#### Check for missing values and data types to understand the structure of the dataset


We use info() method

In [4]:
print("\nDataset Information:")
ad_clicks.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             1654 non-null   object
 1   utm_source          1654 non-null   object
 2   day                 1654 non-null   object
 3   ad_click_timestamp  565 non-null    object
 4   experimental_group  1654 non-null   object
dtypes: object(5)
memory usage: 64.7+ KB


##### Let´s count the total number of Missing Values

In [5]:
print("\nMissing Values Count:")
display(ad_clicks.isnull().sum())


Missing Values Count:


user_id                  0
utm_source               0
day                      0
ad_click_timestamp    1089
experimental_group       0
dtype: int64



We see that the column "ad_click_timestamp" has 1089 missing values

### 3. Data Transformation: Understanding Advertisement Sources

##### Group by utm_source (ad source) to count the number of users associated with each source


In [6]:
source_counts = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print("\nUser counts by UTM source:")
display(source_counts)


User counts by UTM source:


,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


*Conclusion:* the most important source for us would be Google because most of the subscribers came from it; second, we have Facebook. Any Marketing campaign should focus on both of them and maybe to detect the reasons of the low number with the other two sources: twitter and email marketing.

### 4. Feature Engineering: Identifying Clicks

##### Add a new column 'is_click' to indicate whether the ad was clicked (non-null ad_click_timestamp)


In [7]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

##### Analyze clicks by source


In [8]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print("\nClicks by source:")
display(clicks_by_source)


Clicks by source:


,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


#### Pivot data to calculate click percentages by source


In [9]:
clicks_pivot = clicks_by_source.pivot(columns='is_click', index='utm_source', values='user_id').reset_index()
display(clicks_pivot)

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


##### Add a new column for percentage of clicks to our recent created pivoted table


In [10]:
clicks_pivot['percent_clicked'] = clicks_pivot.get(True, 0) / (clicks_pivot.get(True, 0) + clicks_pivot.get(False, 0))
print("\nClick percentages by source:")
display(clicks_pivot)


Click percentages by source:


is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


### 5. Analyze Experiment Groups


##### Group by experimental group to count users


In [11]:
ad_counts = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print("\nUser counts by experimental group:")
display(ad_counts)


User counts by experimental group:


,experimental_group,user_id
0,A,827
1,B,827


##### Group by experimental_group and is_click to analyze click behavior within experimental groups


In [12]:
clicks_by_group = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
display(clicks_by_group)

,experimental_group,is_click,user_id
0,A,False,517
1,A,True,310
2,B,False,572
3,B,True,255


##### Pivot the data to calculate click percentages for each group


In [13]:
clicks_pivot_group = clicks_by_group.pivot(index='experimental_group', columns='is_click', values='user_id').reset_index()
display(clicks_pivot_group)

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


##### Add percentage of users who clicked for each group


In [14]:
clicks_pivot_group['percent_clicked'] = clicks_pivot_group.get(True, 0) / (clicks_pivot_group.get(True, 0) + clicks_pivot_group.get(False, 0)) *100
print("\nClick percentages by experimental group:")
display(clicks_pivot_group)


Click percentages by experimental group:


is_click,experimental_group,False,True,percent_clicked
0,A,517,310,37.484885
1,B,572,255,30.834341


### 6. Analyze Daily Trends in Experimental Groups

##### Filter data for group 'A' and 'B' and have two subsets to use them as variables later


In [15]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

###### Group 'A' analysis


In [16]:
a_clicks_by_day = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
a_clicks_pivot = a_clicks_by_day.pivot(index='day', columns='is_click', values='user_id').reset_index()
a_clicks_pivot['percent_clicked'] = a_clicks_pivot.get(True, 0) / (a_clicks_pivot.get(True, 0) + a_clicks_pivot.get(False, 0))

print("\nGroup A click percentages by day:")
display(a_clicks_pivot)


Group A click percentages by day:


is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


###### Group 'B' analysis

In [17]:
b_clicks_by_day = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
b_clicks_pivot = b_clicks_by_day.pivot(index='day', columns='is_click', values='user_id').reset_index()
b_clicks_pivot['percent_clicked'] = b_clicks_pivot.get(True, 0) / (b_clicks_pivot.get(True, 0) + b_clicks_pivot.get(False, 0))

print("\nGroup B click percentages by day:")
display(b_clicks_pivot)


Group B click percentages by day:


is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


### 7. Identify days with the highest click-through rates (CTR) for both groups

In [20]:
print("\nTop Days by Click-Through Rate for Group A:")
display(a_clicks_pivot.sort_values(by='percent_clicked', ascending=False).head())

print("\nTop Days by Click-Through Rate for Group B:")
display(b_clicks_pivot.sort_values(by='percent_clicked', ascending=False).head())


Top Days by Click-Through Rate for Group A:


is_click,day,False,True,percent_clicked
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
6,7 - Sunday,66,43,0.394495
5,6 - Saturday,73,45,0.381356
0,1 - Monday,70,43,0.380531



Top Days by Click-Through Rate for Group B:


is_click,day,False,True,percent_clicked
1,2 - Tuesday,74,45,0.378151
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927
4,5 - Friday,90,38,0.296875
0,1 - Monday,81,32,0.283186


*Conlusion:* for group A the best days is the Thursday, while for group B the day with most of the clicks is Tuesday. 

### 8. Data Aggregation: Calculate overall CTR


In [21]:
overall_ctr = ad_clicks['is_click'].mean()
print(f"\nOverall Click-Through Rate (CTR): {overall_ctr:.2%}")


Overall Click-Through Rate (CTR): 34.16%
